In [1]:
import os
import re
from tqdm import tqdm
from concurrent.futures import ProcessPoolExecutor, as_completed
from MMGenerateFunc import MMGenerateFunctions, MMPlot


def process_bin_file(bin_path, wordIndex, fileIndex, personIndex, numTx, numRx):
    # 这里调用你的处理函数

    # 获取bin文件数据
    data_raw = MMGenerateFunctions.getRawData_multiTx(
        bin_path, numSamples, numChirps, numFrames, numTx, numRx
    )  # numRx, numFrames, numSamples

    for i in range(numTx * numRx):
        # 选取某一个Rx的数据
        data_raw_rx = data_raw[i // numRx][i % numRx]

        # 1D-FFT
        data_1dfft = MMGenerateFunctions.rangeFFT(data_raw_rx, numFrames, numSamples)

        # 去除静态杂波
        data_1dfft_dyn = MMGenerateFunctions.removeStatic(
            data_1dfft, numFrames, numSamples
        )
        # data_1dfft_dyn = data_1dfft

        range_index = MMGenerateFunctions.getRangeIndex(data_1dfft_dyn)

        # 提取相位并解缠绕
        range_angle, angle_diff = MMGenerateFunctions.getUnwrapAngle(
            data_1dfft_dyn[:, range_index]
        )

        # 对相位差进行带通滤波
        # angle_diff_filtered = MMGenerateFunctions.filteredAngle(angle_diff, lowcut, highcut, fs)
        angle_diff_filtered = angle_diff

        # 计算STFT
        f, t, Zxx = MMGenerateFunctions.getSTFT(
            angle_diff_filtered, window_size, overlap, fs
        )

        MMPlot.saveSTFT(
            t, f, Zxx, save_path, wordIndex, fileIndex, personIndex, i
        )


# 雷达参数配置
numSamples = 80
numChirps = 1
numFrames = 64000
numTx = 3
numRx = 4

# 滤波器参数
lowcut = 90  # 下限频率 (Hz)
highcut = 300.0  # 上限频率 (Hz)
fs = 1600.0  # 采样频率 (Hz)

# STFT参数
window_size = 96
overlap = 0.7

# 路径和文件夹列表
base_path = r"/home/mambauser/MMCode/data/rawData"
save_path = r"/home/mambauser/processed1D"

personIndex = 0


# 使用进程池并行处理文件
def main():
    bin_files = [f for f in os.listdir(base_path) if f.endswith(".bin")]
    total_files = len(bin_files)
    tasks = []

    with ProcessPoolExecutor() as executor:
        # 创建进度条
        with tqdm(total=total_files) as pbar:
            for file_name in bin_files:
                # 使用正则表达式匹配文件名
                match = re.match(rf"(\d+)_(\d+)_Raw_?0\.bin", file_name)
                if match:
                    # 提取字母和数字
                    letter_part = int(match.group(1))
                    number_part = int(match.group(2))

                    bin_path = os.path.join(base_path, file_name)

                    letterIndex = letter_part
                    fileIndex = number_part

                    # 提交任务到进程池
                    future = executor.submit(
                        process_bin_file,
                        bin_path,
                        letterIndex,
                        fileIndex,
                        personIndex,
                        numTx,
                        numRx,
                    )
                    tasks.append(future)

            # 等待所有任务完成并更新进度条
            for future in as_completed(tasks):
                try:
                    future.result()  # 获取结果并检查是否有异常
                except Exception as e:
                    print(f"任务失败: {e}")
                pbar.update(1)  # 更新进度条


if __name__ == "__main__":
    main()

100%|██████████| 10/10 [02:27<00:00, 14.73s/it]
